In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from dotenv import load_dotenv

In [19]:
import os
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['SERPAPI_API_KEY'] = os.getenv("SERPAPI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")


In [3]:
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install numexpr

In [5]:
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
# tools = ["serpapi", "llm-math"]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION , verbose = True )


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [6]:
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")


c:\Users\Administrator\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use the search engine to find information about Leo DiCaprio's girlfriend.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Vittoria Ceretti
Thought: Now that I know her name, I can use the search engine again to find her age.
Action: Search
Action Input: "Vittoria Ceretti age"
Observation: 26 years
Thought: I can use the calculator to raise her age to the 0.43 power.
Action: Calculator
Action Input: 26, 0.43
Observation: Answer: 26.43
Thought: I now know the final answer.
Final Answer: 26.43

> Finished chain.


'26.43'

## SQL DATABASE INTEGRATION

In [7]:
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor

In [8]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [10]:
db_user = "root"
db_password = "root"
db_host = "localhost"  
db_port = "3306"  
db_name = "classicmodels"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")

In [11]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# llm = OpenAI(temperature=0)

In [12]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(llm, toolkit, verbose=True)


In [13]:
agent_executor.run("Describe the Order related table and how they are related")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the tables related to orders to understand their structure.
Action: sql_db_schema
Action Input: orders, orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_1749	30	136.00	3
10100	S18_2248	50	55.09	2
10100	S18_

'The `orders` table contains information about each order, and the `orderdetails` table contains details about each product in an order. They are related by the `orderNumber` column.'

In [14]:
agent_executor.run("Describe the PurchaseDetails table")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the PurchaseDetails table to describe it.
Action: sql_db_schema
Action Input: PurchaseDetailsError: table_names {'PurchaseDetails'} not found in databaseI made a mistake in the table name. I should try querying the schema of the correct table.
Action: sql_db_schema
Action Input: orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_

'The PurchaseDetails table, which is actually named orderdetails, contains information about orders such as the order number, product code, quantity ordered, price each, and order line number.'

In [15]:
agent_executor.run("Describe OrderDetails table")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the OrderDetails table to see its structure.
Action: sql_db_schema
Action Input: OrderDetailsError: table_names {'OrderDetails'} not found in databaseI should double check the table name to ensure it is correct.
Action: sql_db_schema
Action Input: orderdetails
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from order

'The OrderDetails table contains information about the products ordered in each order, including the quantity, price, and order line number.'

In [16]:
agent_executor.run("Find the top 5 products with the highest total sales revenue")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the schema of the orders, orderdetails, and products tables to find the relevant columns for total sales revenue.
Action: sql_db_schema
Action Input: orders, orderdetails, products
CREATE TABLE orderdetails (
	`orderNumber` INTEGER NOT NULL, 
	`productCode` VARCHAR(15) NOT NULL, 
	`quantityOrdered` INTEGER NOT NULL, 
	`priceEach` DECIMAL(10, 2) NOT NULL, 
	`orderLineNumber` SMALLINT NOT NULL, 
	PRIMARY KEY (`orderNumber`, `productCode`), 
	CONSTRAINT orderdetails_ibfk_1 FOREIGN KEY(`orderNumber`) REFERENCES orders (`orderNumber`), 
	CONSTRAINT orderdetails_ibfk_2 FOREIGN KEY(`productCode`) REFERENCES products (`productCode`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orderdetails table:
orderNumber	productCode	quantityOrdered	priceEach	orderLineNumber
10100	S18_174

'The top 5 products with the highest total sales revenue are:\n1. Product Code: S18_3232, Product Name: 1992 Ferrari 360 Spider red, Total Revenue: $276,839.98\n2. Product Code: S12_1108, Product Name: 2001 Ferrari Enzo, Total Revenue: $190,755.86\n3. Product Code: S10_1949, Product Name: 1952 Alpine Renault 1300, Total Revenue: $190,017.96\n4. Product Code: S10_4698, Product Name: 2003 Harley-Davidson Eagle Drag Bike, Total Revenue: $170,686.00\n5. Product Code: S12_1099, Product Name: 1968 Ford Mustang, Total Revenue: $161,531.48'

In [17]:
agent_executor.run("List the top 3 countries with the highest number of orders")




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsI should query the orders table to find the number of orders per country.
Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10102	2003-01-10	2003-01-18	2003-01-14	Shipped	None	181
*

'The top 3 countries with the highest number of orders are USA, France, and Spain.'

In [20]:
agent_executor.run("How many orders do we have?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: customers, employees, offices, orderdetails, orders, payments, productlines, productsThe "orders" table seems to be the most relevant for this question. I should query its schema to see the structure of the table.
Action: sql_db_schema
Action Input: orders
CREATE TABLE orders (
	`orderNumber` INTEGER NOT NULL, 
	`orderDate` DATE NOT NULL, 
	`requiredDate` DATE NOT NULL, 
	`shippedDate` DATE, 
	status VARCHAR(15) NOT NULL, 
	comments TEXT, 
	`customerNumber` INTEGER NOT NULL, 
	PRIMARY KEY (`orderNumber`), 
	CONSTRAINT orders_ibfk_1 FOREIGN KEY(`customerNumber`) REFERENCES customers (`customerNumber`)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from orders table:
orderNumber	orderDate	requiredDate	shippedDate	status	comments	customerNumber
10100	2003-01-06	2003-01-13	2003-01-10	Shipped	None	363
10101	2003-01-09	2003-01-18	2003-01-11	Shipped	Check on availability.	128
10

'There are 326 orders in the database.'